In [1]:
!conda install -y gdown
print('Done')
RNN= None
CNN= 1
LSTM= None

In [2]:
    
import gdown

url =  'https://drive.google.com/file/d/1qvqsWCOjcoTkbAG7MZcPrxZPucqdYR9Q/view?usp=sharing'
Id = '1qvqsWCOjcoTkbAG7MZcPrxZPucqdYR9Q'
output = 'data.dat'

gdown.download(id=Id, output=output, quiet=False)
print('Done')

# data preparation

In [17]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import scipy as sp

In [11]:
f = open('data.dat','rb')
data = pickle.load(f, encoding='latin1')

In [12]:
data.keys()

In [13]:
def truncateX(dataX,newData):
    if dataX is None:
        dataX= data[key]
    else:
        dataX= np.vstack((dataX,data[key]))
    return dataX

In [14]:
dataX= None
dataY= []
for key in data:
    size= data[key].shape[0]
    for i in range (size):
        dataY.append(key[0]+" "+ str(key[1])) 
    dataX=truncateX(dataX,data[key])   

In [15]:
print(f'{len(dataX)}\t{len(dataY)}')
print(np.unique(dataY, return_counts=True))

In [18]:
derv_X = np.apply_along_axis(lambda column:np.gradient(column),2,dataX)
intg_X = np.apply_along_axis(lambda column:np.hstack((0,column)),2,np.apply_along_axis(lambda column:sp.integrate.cumtrapz(column,dx=1,axis=-1),2,dataX))

# visualize the data

In [19]:
while True:
    y= int(input(''))
    if y ==-1:
        break
    plt.plot(dataX[y,0])
    plt.plot(dataX[y,1])
    plt.show()

# data splitting

In [20]:
dataY= np.array(dataY)
trainX,testX,trainY,testY = train_test_split(dataX,dataY, test_size=0.3,random_state=42,stratify=dataY)

trainX, valX, trainY, valY = train_test_split(trainX, trainY, random_state=42, test_size=0.05, shuffle=True,stratify=trainY)
print(f'{trainX.shape}\t{testX.shape}\t{valX.shape}\t{valY.shape}\t{trainY.shape}\t{testY.shape}')
trainXX = trainX

In [21]:
print(np.unique(trainY, return_counts=True))
print(np.unique(testY, return_counts=True))
print(np.unique(valY, return_counts=True))

In [22]:
def return_labels(st):
    return st.split(" ")[0]

labels = trainY.copy()
returnlabels = np.vectorize(return_labels)
labels = returnlabels(labels)

np.unique(labels, return_counts=True)

In [23]:
encoder = OneHotEncoder()
trainYY = encoder.fit_transform(labels.reshape((-1,1))).toarray()
valYY = encoder.fit_transform(returnlabels(valY).reshape((-1,1))).toarray()
testYY = encoder.fit_transform(returnlabels(testY).reshape((-1,1))).toarray()

In [26]:
for i in range(5):
    print(labels[i])
    print(trainYY[i])
    

# traing our models 

In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.compat.v1.keras import layers
from tensorflow.keras.layers import Conv1D,Conv2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization
gpus = tf.config.list_physical_devices('GPU')
gpus

# the following are the 2 best networks that had high accuary from what we trained we trained

In [27]:
#from paper Convolutional Radio Modulation Recognition Networks
def getCNNModelbestScore():
    model= models.Sequential()
#     model.add(Reshape((2,128,1)), input_shape=(2,128,1))
    model.add(BatchNormalization(input_shape=(2,128,1)))
    model.add(Conv2D(filters=256,kernel_size=(1,3),padding='Same',strides=(1),activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.6))
    model.add(Conv2D(filters=80,kernel_size = (2,3),padding='Same',strides=(1),activation='relu'))
    model.add(Dropout(0.6))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    return model

In [29]:
#from paper Deep Architectures for Modulation Recognition
def getCNNModel2ndbestScore():
    model= models.Sequential()
#     model.add(Reshape((2,128,1)), input_shape=(2,128,1))
    model.add(BatchNormalization(input_shape=(2,128,1)))
    model.add(Conv2D(filters=50,kernel_size=(1,8),padding='Same',strides=(1),activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Conv2D(filters=50,kernel_size = (1,8),padding='Same',strides=(1),activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    return model

# Conv LSTM (we tried conv lstm as mentioned in bonus section)

In [30]:
#from paper Deep Architectures for Modulation Recognition
def getCLDNN():
    model_input = keras.Input(((2,128,1)),  name="mod")
    x = layers.Conv2D(50, kernel_size=(1,8), strides=(1),padding="same", activation="relu")(model_input)
    x_skip = x
    x = layers.Conv2D(50, kernel_size=(1,8), strides=(1),padding="same", activation="relu")(x)
    x = layers.Conv2D(50, kernel_size=(1,8), strides=(1),padding="same", activation="relu")(x)
    x = layers.Add()([x, x_skip])
    x = layers.Reshape(target_shape=(2,-1))(x)
    x = layers.SimpleRNN(50)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    model_output = layers.Dense(10, activation="softmax")(x)
    model = keras.Model(model_input, model_output, name="CLDNN")
    (model.summary())
    return model

In [32]:
#achieved very bad score
def ConvLSTM2DModel():
    model=models.Sequential()
    model.add(keras.Input(shape=(1,2,128,1)))
    model.add(layers.ConvLSTM2D(50,(3,3)))
    model.add(Flatten())
    model.add(layers. Dense (128,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    return model

# 3 required architecture 

In [33]:
def getCNNModel():
    model= models.Sequential()
    model.add(Conv2D(filters=64,kernel_size=(1,3),padding='Same',strides=(1),activation='relu',input_shape=((2,128,1))))
    model.add(Conv2D(filters=16,kernel_size = (2,3),padding='Same',strides=(1),activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    return model
def getRNNModel():
    model=models.Sequential()
    model.add(keras.Input(shape=(2,128)))
    model.add(BatchNormalization())
    model.add(layers. SimpleRNN (128))
    model.add(Dropout(0.5))
    # model.add(layers. Dense (128,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    return model
def getLSTMModel():
    model=models.Sequential()
    model.add(keras.Input(shape=(2,128)))
    model.add(BatchNormalization())
    model.add(layers. LSTM (128))
    model.add(Dropout(0.5))
    # model.add(layers. Dense (128,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    return model

In [34]:
model= None
if RNN is not None:
    model= getRNNModel()
elif CNN is not None:
    trainXX = trainXX.reshape((-1,2,128,1))
    valX = valX.reshape((-1,2,128,1)) 
    testX = testX.reshape((-1,2,128,1))
#     trainXX = trainXX.reshape((-1,1,2,128,1))# ConvLSTM2D
#     valX = valX.reshape((-1,1,2,128,1)) 
#     testX = testX.reshape((-1,1,2,128,1))
    model= getCNNModel()
elif LSTM is not None:
    model= getLSTMModel()

In [35]:
opt = keras.optimizers.Adam(learning_rate=0.01)

optt=keras.optimizers.Adagrad(
    learning_rate=0.01,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
)

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model=models.load_model('../input/runs12345/modelconv.h5')
epoch= 50
learningratereduction = ReduceLROnPlateau(monitor='val_acc', patience=2,verbose=1, factor=0.1, min_lr=0.00001,min_delta=0.01,)
earlystopping = EarlyStopping(monitor ="val_acc", patience = 5, restore_best_weights = True)

history = model.fit(trainXX,trainYY , epochs=epoch,batch_size=256 ,
                    validation_data=(valX,valYY),callbacks=[earlystopping,learningratereduction])

1

# test our model

In [38]:
val_loss=None
train_loss=None
val_acc=None
train_acc= None

if val_loss is None:
    val_loss=history.history['val_loss']
    train_loss=history.history['loss']
    val_acc=history.history['val_accuracy']
    train_acc=history.history['accuracy']
else:
    val_loss=np.hstack((val_loss,history.history['val_loss']))
    train_loss=np.hstack((train_loss,history.history['loss']))
    val_acc=np.hstack((val_acc,history.history['val_accuracy']))
    train_acc=np.hstack((train_acc,history.history['accuracy']))

model.save('model.h5')

In [39]:
x= np.arange(0,len(val_loss),1)
plt.plot(x,val_loss,label='Val_loss', color='blue')
plt.plot(x,train_loss,label='Train_Loss', color='red')
plt.legend()
plt.xlim(0,len(val_loss))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()


In [40]:
x= np.arange(0,len(val_loss),1)
plt.plot(x,val_acc,label='Val_ACC', color='blue')
plt.plot(x,train_acc,label='Train_ACC', color='red')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('ACC')
plt.show()

In [42]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,title='Confusion matrix',cmap=plt.cm.Blues):
    plt.figure(figsize=(20,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
y_pred = model.predict(testX)
y_pred = np.argmax(y_pred, axis=1)
y_check = np.argmax(testYY, axis=1)
cm = confusion_matrix(y_true=y_check, y_pred=y_pred)
cm_plot_labels = ['8PSK', 'AM-DSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16','QAM64', 'QPSK', 'WBFM']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [43]:
def return_SNR(st):
    return int(st.split(" ")[1])

SNR = testY.copy()
returnlSNR = np.vectorize(return_SNR)
SNR = returnlSNR(SNR)

np.unique(SNR, return_counts=True)

acc_snr = []
i_snr = []
for i in range(-20,20,2):
    score, acc = model.evaluate(testX[(SNR) == i], testYY[(SNR) == i],batch_size=128)
    acc_snr.append(acc)
    i_snr.append(i)

In [44]:
acc_snr = []
i_snr = []
for i in range(-20,20,2):
    score, acc = model.evaluate(testX[(SNR) == i], testYY[(SNR) == i],batch_size=128)
    acc_snr.append(acc)
    i_snr.append(i)

In [45]:
plt.plot(i_snr,acc_snr,label='Val_ACC', color='blue')
plt.legend()
plt.xlabel('SNR')
plt.ylabel('ACC')
plt.show()

In [46]:
score, acc = model.evaluate(testX[(SNR) == 0], testYY[(SNR) == 0],batch_size=128)
print('Test score:', score)
print('Test accuracy:', acc)

# other networks we tried

In [ ]:
def getCNNModelv3():
    model= models.Sequential()
    model.add(Conv2D(filters=60,kernel_size=(1,8),padding='Same',strides=(1),activation='relu',input_shape=((2,128,1))))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=60,kernel_size = (1,8),padding='Same',strides=(1),activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    return model

In [ ]:
def getCNNModelv8():
    model= models.Sequential()
#     model.add(Reshape((2,128,1)), input_shape=(2,128,1))
    model.add(BatchNormalization(input_shape=(2,128,1)))
    model.add(Conv2D(filters=256,kernel_size=(1,3),padding='Same',strides=(1),activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.6))
    model.add(Conv2D(filters=80,kernel_size = (2,3),padding='Same',strides=(1),activation='relu'))
    model.add(Dropout(0.6))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    return model

In [ ]:
def getCNNModelv4():
    model= models.Sequential()
    model.add(Conv2D(filters=128,kernel_size=(1,8),padding='Same',strides=(1),activation='relu',input_shape=((2,128,1))))
#     model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=80,kernel_size = (2,8),padding='Same',strides=(1),activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    return model

In [ ]:
def getCNNModelv2():
    model= models.Sequential()
    model.add(Conv2D(filters=128,kernel_size=(1,3),padding='Same',strides=(1),activation='relu',input_shape=((2,128,1))))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=80,kernel_size = (2,3),padding='Same',strides=(1),activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    return model

In [ ]:
def getCNNModelv3():
    model= models.Sequential()
    model.add(Conv2D(filters=60,kernel_size=(1,8),padding='Same',strides=(1),activation='relu',input_shape=((2,128,1))))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=60,kernel_size = (1,8),padding='Same',strides=(1),activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    return model

In [ ]:
def getCNNModelv7():
    model= models.Sequential()
    model.add(Conv2D(filters=128,kernel_size=(1,3),padding='Same',strides=(1),activation='relu',input_shape=((2,128,1))))
#     model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=80,kernel_size = (2,3),padding='Same',strides=(1),activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    return model